In [207]:
%config Completer.use_jedi = False

In [208]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
from datetime import datetime

In [209]:
# Caminhos dos diretórios raiz
# general_path = 'C:/Users/cnalm/OneDrive/Hidroweb'
# general_path = 'D:/Dados_Nuvem/OneDrive/Hidroweb'
general_path = r'C:\Users\linde\OneDrive\Hidroweb'
idw_path = general_path + r'\Consolidated Files\NetCDF\IDW'
cn_path_station = r'C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NormaisClimatologicas\Estacoes'
cn_path_prec = r'C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NormaisClimatologicas\Prec'

In [210]:
all_files = glob.glob(idw_path + "\*.nc")
total_length = len(all_files)
print(total_length)
all_files[10957]

21915


'C:\\Users\\linde\\OneDrive\\Hidroweb\\Consolidated Files\\NetCDF\\IDW\\precipitation_idw_1991-01-01.nc'

In [211]:
def convert_degrees(string):
    parts = string.split('°')
    degrees = float(parts[0])
    minutes = float(parts[1].strip("'").split('\'')[0])
    direction = parts[1].strip("'").split('\'')[1]
    
    decimal_degrees = degrees + minutes / 60
    
    if direction == 'S' or direction == 'W':
        decimal_degrees *= -1
    
    return decimal_degrees

In [212]:
df_cn_station = pd.read_excel(cn_path_station + "/NC_ESTACOES_1991_2020.xlsx")
df_cn_station = df_cn_station[["Código","Nome",		"UF",	"Latitude",	"Longitude",	"Altitude (m)"]]
# df_cn_station_1961_1990['Latitude_Decimal'] = df_cn_station_1961_1990['Latitude'].apply(convert_degrees)
# df_cn_station_1961_1990['Longitude_Decimal'] = df_cn_station_1961_1990['Longitude'].apply(convert_degrees)
df_cn_station.columns = ['Código', 'Nome', 'UF', 'Latitude_Decimal', 'Longitude_Decimal', 'Altitude (m)']
df_cn_station

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m)
0,82704,CRUZEIRO DO SUL,AC,-7.610736,-72.681324,214.19
1,82915,RIO BRANCO,AC,-9.959167,-67.868889,160.71
2,82807,TARAUACA,AC,-8.160278,-70.768611,172.27
3,82989,AGUA BRANCA,AL,-9.264722,-37.938056,603.42
4,82994,MACEIO,AL,-9.551389,-35.770833,84.12
...,...,...,...,...,...,...
266,83033,PALMAS,TO,-10.190897,-48.301822,291.68
267,82863,PEDRO AFONSO,TO,-8.968611,-48.177222,189.53
268,83228,PEIXE,TO,-12.015278,-48.545000,252.24
269,83064,PORTO NACIONAL,TO,-10.710833,-48.406389,243.28


In [213]:
def pr_value_calc(lat, lon, df):
    df_temp = df.sel(Longitude = lon, Latitude = lat, method = 'nearest')
    return df_temp.variables['Precipitation'].to_numpy()

In [214]:
count = 0
total_len = 0
df_result = []
for filename in all_files[10957:]:
    print(count, filename)
    count = count + 1
    date_str = filename[-13:-3]
    ref_date = datetime.strptime(date_str, "%Y-%m-%d")
    df_temp = df_cn_station.copy(deep=True)
    lat_list = df_temp['Latitude_Decimal'].values.tolist()
    lon_list = df_temp['Longitude_Decimal'].values.tolist()
    coord_list = []
    for i in range(len(lat_list)):
        # print(lat_list[i])
        coord_list.append([lat_list[i], lon_list[i]])
    ds = xr.open_dataset(filename)
    pr_list = []
    for coord in coord_list:
        pr_value = pr_value_calc(coord[0], coord[1], ds)
        pr_list.append(pr_value[0])
        # print(pr_value)
    # print(pr_list)
    df_temp['Value_Interpolated'] = pr_list
    df_temp['Date'] = ref_date
    # print(df_temp.head(5))
    if len(df_result) == 0:
        df_result = df_temp.copy(deep=True)
    else:
        df_result = pd.concat([df_result, df_temp], ignore_index= True)
    total_len = total_len + len(df_temp)
df_result

0 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-01.nc


1 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-02.nc
2 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-03.nc
3 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-04.nc
4 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-05.nc
5 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-06.nc
6 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-07.nc
7 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-08.nc
8 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-09.nc
9 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-10.nc
10 C:\Users\linde\OneDrive\Hidroweb\Consolidated Files\NetCDF\IDW\precipitation_idw_1991-01-11.nc
11 C:\Users\linde\OneDrive\Hi

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),Value_Interpolated,Date
0,82704,CRUZEIRO DO SUL,AC,-7.610736,-72.681324,214.19,0.023620,1991-01-01
1,82915,RIO BRANCO,AC,-9.959167,-67.868889,160.71,8.036516,1991-01-01
2,82807,TARAUACA,AC,-8.160278,-70.768611,172.27,0.002238,1991-01-01
3,82989,AGUA BRANCA,AL,-9.264722,-37.938056,603.42,0.470556,1991-01-01
4,82994,MACEIO,AL,-9.551389,-35.770833,84.12,1.520571,1991-01-01
...,...,...,...,...,...,...,...,...
2969613,83033,PALMAS,TO,-10.190897,-48.301822,291.68,1.007039,2020-12-31
2969614,82863,PEDRO AFONSO,TO,-8.968611,-48.177222,189.53,17.001843,2020-12-31
2969615,83228,PEIXE,TO,-12.015278,-48.545000,252.24,0.000000,2020-12-31
2969616,83064,PORTO NACIONAL,TO,-10.710833,-48.406389,243.28,0.000000,2020-12-31


In [215]:
df_check = df_result[pd.isna(df_result['Value_Interpolated'])]
df_check = df_check[['Código', 'Nome', 'UF', 'Latitude_Decimal', 'Longitude_Decimal']].drop_duplicates(ignore_index=True)
df_check

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal


In [216]:
df_result['Date'] = pd.to_datetime(df_result['Date'])
df_result['Year'] = df_result['Date'].dt.year
df_result['Month'] = df_result['Date'].dt.month
df_result['Date'] = df_result['Date'].astype(str)
df_result 

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),Value_Interpolated,Date,Year,Month
0,82704,CRUZEIRO DO SUL,AC,-7.610736,-72.681324,214.19,0.023620,1991-01-01,1991,1
1,82915,RIO BRANCO,AC,-9.959167,-67.868889,160.71,8.036516,1991-01-01,1991,1
2,82807,TARAUACA,AC,-8.160278,-70.768611,172.27,0.002238,1991-01-01,1991,1
3,82989,AGUA BRANCA,AL,-9.264722,-37.938056,603.42,0.470556,1991-01-01,1991,1
4,82994,MACEIO,AL,-9.551389,-35.770833,84.12,1.520571,1991-01-01,1991,1
...,...,...,...,...,...,...,...,...,...,...
2969613,83033,PALMAS,TO,-10.190897,-48.301822,291.68,1.007039,2020-12-31,2020,12
2969614,82863,PEDRO AFONSO,TO,-8.968611,-48.177222,189.53,17.001843,2020-12-31,2020,12
2969615,83228,PEIXE,TO,-12.015278,-48.545000,252.24,0.000000,2020-12-31,2020,12
2969616,83064,PORTO NACIONAL,TO,-10.710833,-48.406389,243.28,0.000000,2020-12-31,2020,12


In [217]:
df_result.to_hdf(cn_path_prec + '/BRASIL_CN_INTERPOLATED_VALUES_1991_2020.h5', 'table_data', mode = 'w', append = False, complevel = 9, encoding="cp860")

C:\Users\linde\AppData\Local\Temp\ipykernel_22908\2610448882.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  df_result.to_hdf(cn_path_prec + '/BRASIL_CN_INTERPOLATED_VALUES_1991_2020.h5', 'table_data', mode = 'w', append = False, complevel = 9, encoding="cp860")


In [218]:
df_interpolation = pd.read_hdf(cn_path_prec + '/BRASIL_CN_INTERPOLATED_VALUES_1991_2020.h5', 'table_data')
df_interpolation

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),Value_Interpolated,Date,Year,Month
0,82704,CRUZEIRO DO SUL,AC,-7.610736,-72.681324,214.19,0.023620,1991-01-01,1991,1
1,82915,RIO BRANCO,AC,-9.959167,-67.868889,160.71,8.036516,1991-01-01,1991,1
2,82807,TARAUACA,AC,-8.160278,-70.768611,172.27,0.002238,1991-01-01,1991,1
3,82989,AGUA BRANCA,AL,-9.264722,-37.938056,603.42,0.470556,1991-01-01,1991,1
4,82994,MACEIO,AL,-9.551389,-35.770833,84.12,1.520571,1991-01-01,1991,1
...,...,...,...,...,...,...,...,...,...,...
2969613,83033,PALMAS,TO,-10.190897,-48.301822,291.68,1.007039,2020-12-31,2020,12
2969614,82863,PEDRO AFONSO,TO,-8.968611,-48.177222,189.53,17.001843,2020-12-31,2020,12
2969615,83228,PEIXE,TO,-12.015278,-48.545000,252.24,0.000000,2020-12-31,2020,12
2969616,83064,PORTO NACIONAL,TO,-10.710833,-48.406389,243.28,0.000000,2020-12-31,2020,12
